## papers with code 원본 불러오기

In [ ]:
paperswithcode = pd.read_csv("/content/drive/MyDrive/EDA 3조(code)/paperswithcode_fin.csv")

In [ ]:
paperswithcode

## 모델에서 특수문자 regex에서 except처리

In [ ]:
def pattern_change(data):
  # print(data.group())
  if data.group() == ")":
    return "\)"
  elif data.group() == "(":
    return "\("
  elif data.group() == "+":
    return "\+"

In [ ]:
for i in tqdm(paperswithcode.index):
  keyword = paperswithcode.iloc[i,2]
  if type(keyword) != type(1.0):
    keyword = re.sub("\(|\)|\+", pattern_change, keyword)
    paperswithcode.iloc[i,2] = keyword

100%|██████████| 1928/1928 [00:00<00:00, 10885.09it/s]


## 모델에 해당하는 논문 index 추출

In [ ]:
def categorize(data, keyword):
  return data[data["title"].str.contains(f"\W\d*\s*{keyword}s?\s\d\W",case=False,regex=True) | data["abstract"].str.contains(f"\W\d*\s*{keyword}s?\s*\d*\W",case=False,regex=True)]

In [ ]:
# 2시간...
list_all = []
for i in tqdm(paperswithcode.index):
  keyword = paperswithcode.iloc[i,2]
  ddd = categorize(data_ai, keyword)
  if len(ddd.index) > 0:
    list_all.append(
        str(set(ddd.index)).replace("{","").replace("}","")
    )
  else:
    list_all.append(None)

100%|██████████| 1928/1928 [4:11:55<00:00,  7.84s/it]


## 추출한 index paperswithcode.csv에 병합, 저장

In [ ]:
paperswithcode['indexs'] = list_all
paperswithcode

,method_name,categories_name,model_name,indexs
0,Computer Vision,Image Models,ResNet,"32768, 106502, 81929, 106505, 155660, 172046, ..."
1,Computer Vision,Image Models,Vision Transformer,"135170, 34820, 118792, 112653, 135184, 24594, ..."
2,Computer Vision,Image Models,VGG,"90113, 88070, 106505, 38922, 104461, 120846, 8..."
3,Computer Vision,Image Models,DenseNet,"94210, 96259, 86021, 92178, 143379, 98326, 983..."
4,Computer Vision,Image Models,VGG-16,"90113, 111619, 110086, 86537, 38922, 89611, 99..."
...,...,...,...,...
1923,Graphs,Graph Representation Learning,L-GCN,60984
1924,Graphs,Graph Representation Learning,GFSA,159177
1925,Graphs,Graph Representation Learning,DyGED,170365
1926,Graphs,Graph Representation Learning,DeepDrug,None


In [ ]:
paperswithcode.to_csv("/content/drive/MyDrive/EDA 3조(code)/paperswithcode_indexs_fin2.csv", index = False)

## paperswithcode.csv 저장파일 불러오기

In [ ]:
# paperswithcode = pd.read_csv("/content/drive/MyDrive/EDA 3조(code)/paperswithcode_indexs_fin.csv")
paperswithcode = pd.read_csv("/content/drive/MyDrive/EDA 3조(code)/치웅님 서치/paperswithcode_indexs_fin2.csv")

## paperswithcode.csv를 중분류에 해당하는 dictionary 분류

In [ ]:
def small_cate_to_middle_cate(data):
  category_name = None
  category_list = []
  total_dict = {}
  index_num = None
  for index, i in enumerate(data.iloc[:,3]):
    if data.iloc[index,1] != category_name:
      if len(category_list) > 0:
        print(category_name)
        print(category_list)
        total_dict[category_name] = category_list
      category_name = data.iloc[index,1]
      category_list = []
      index_num = index
    list_data=None
    if type(i) != type(1.0) and i:
      list_data = i.split(", ")
      category_list +=list_data
  print(category_name)
  print(category_list)
  total_dict[category_name] = category_list
  return total_dict

In [ ]:
middle_dict = small_cate_to_middle_cate(paperswithcode)

## paperswithcode.csv에서 중분류에 해당하는 dictionary를 대분류의 dictionary로 분류

In [ ]:
def middel_dict_to_big_dict(data, middle_dict):
  category_name = None
  category_list = []
  total_dict = {}
  data = data.drop_duplicates(['categories_name'], keep = 'first')
  for index, i in enumerate(data.index):
    try:
      if data.iloc[index,0] != category_name:
        if len(category_list) > 0:
          total_dict[category_name] = category_list
        category_name = data.iloc[index,0]
        category_list = []
      # list_data=None
      # print(i)
      # if i:
      # print(middle_dict[data.iloc[index,1]])
      category_list +=middle_dict[data.iloc[index,1]]
    except:
      pass
  total_dict[category_name] = category_list
  return total_dict

In [ ]:
cate_dic = middel_dict_to_big_dict(paperswithcode, middle_dict)


## data_ai를 paperswithcode의 분류법으로 분류 및 저장

In [ ]:
data_ai["is_CV"] = np.nan
data_ai["is_NLP"] = np.nan
data_ai["is_RL"] = np.nan
data_ai["is_Au"] = np.nan
data_ai["is_Se"] = np.nan
data_ai["is_Gr"] = np.nan

In [ ]:
for index, i in enumerate(cate_dic.keys()):
  for j in cate_dic[i]:
    if index == 0:
      # if type(data_ai.loc[j,"is_cv"]) == type(1.0):
      data_ai.loc[int(j),"is_CV"] = True
    elif index == 1:
      data_ai.loc[int(j),"is_NLP"] = True
    elif index == 2:
      data_ai.loc[int(j),"is_RL"] = True
    elif index == 3:
      data_ai.loc[int(j),"is_Au"] = True
    elif index == 4:
      data_ai.loc[int(j),"is_Se"] = True
    elif index == 5:
      data_ai.loc[int(j),"is_Gr"] = True

In [ ]:
data_ai.to_csv("/content/drive/MyDrive/EDA 3조(code)/data_ai_fin2.csv", index = False)